In [1]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import cartopy
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import math as math
import sys
sys.path.insert(1, '/home/sypmauu/GalapagosProject/scripts/functions')
from ParticlePlotFunctions import *
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
# compute 2008 year mean of MITgcm data

Ufiles = '/home/sypmauu/GalapagosProject/data/MIT4km/RGEMS3_2008_Surf.nc'
Xfiles = '/home/sypmauu/GalapagosProject/data/MIT4km/RGEMS3_Surf_grid.nc'
dfile = Dataset(Xfiles)
lon_MIT = dfile.variables['XG'][:]
lat_MIT = dfile.variables['YG'][:]

ddates = range(0,365)

Umean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)
Vmean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)
Tmean = np.zeros([len(lat_MIT),len(lon_MIT)], dtype=float)

teller = 0
dfile = Dataset(Ufiles)
for t in ddates:
    teller += 1
    print(teller)
    Umean += dfile.variables['UVEL'][t,:,:]
    Vmean += dfile.variables['VVEL'][t,:,:]
    Tmean += dfile.variables['THETA'][t,:,:]

Umean_MIT = Umean/teller
Vmean_MIT = Vmean/teller
Tmean_MIT = Tmean/teller

Umean_MIT[Umean_MIT==0]=nan
Vmean_MIT[Vmean_MIT==0]=nan
Tmean_MIT[Tmean_MIT==0]=nan


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
# compute 2008 year mean of CMEMS data

ddir = '/projects/0/topios/hydrodynamic_data/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/'
datafiles = sorted(glob(ddir+'mercatorglorys12v1_gl12_mean_2008*.nc'))

latdim = np.arange(800,1100)
londim = np.arange(700,1500)
dfile = Dataset(datafiles[0])
lon_CMEMS = dfile.variables['longitude'][londim]
lat_CMEMS = dfile.variables['latitude'][latdim]

ddates = range(0,365)

Umean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)
Vmean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)
Tmean = np.zeros([len(lat_CMEMS),len(lon_CMEMS)], dtype=float)

teller = 0
for t in ddates:
    teller += 1
    print(teller)
    dfile = Dataset(datafiles[t])
    Umean += dfile.variables['uo'][0,0,latdim,londim]
    Vmean += dfile.variables['vo'][0,0,latdim,londim]
    Tmean += dfile.variables['thetao'][0,0,latdim,londim]

Umean_CMEMS = Umean/teller
Vmean_CMEMS = Vmean/teller
Tmean_CMEMS = Tmean/teller

Umean_CMEMS[Umean_CMEMS<=-100]=nan
Vmean_CMEMS[Vmean_CMEMS<=-100]=nan
Tmean_CMEMS[Tmean_CMEMS<=-100]=nan

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
# compute 2008 year mean of NEMO

ddir = '/projects/0/topios/hydrodynamic_data/NEMO-MEDUSA/ORCA0083-N006/means/'
ddates = range(584,657) #year 2008
nz = 0
map_crop = [2000, 3001, 1000, 2001]

Field = ReadNemoMean(ddir, ddates, nz, map_crop)

lon_NEMO = Field['lon']
lat_NEMO = Field['lat']
Umean_NEMO = Field['Umean']
Vmean_NEMO = Field['Vmean']
Tmean_NEMO = Field['Tmean']

In [16]:
# save data

np.savez('meanfields2008_MIT.npz',
         lon_MIT = lon_MIT,
         lat_MIT = lat_MIT,
         Umean_MIT = Umean_MIT,
         Vmean_MIT = Vmean_MIT,
         Tmean_MIT = Tmean_MIT)

np.savez('meanfields2008_NEMO.npz',
         lon_NEMO = lon_NEMO,
         lat_NEMO = lat_NEMO,
         Umean_NEMO = Umean_NEMO,
         Vmean_NEMO = Vmean_NEMO,
         Tmean_NEMO = Tmean_NEMO)

np.savez('meanfields2008_CMEMS.npz',
         lon_CMEMS = lon_CMEMS,
         lat_CMEMS = lat_CMEMS,
         Umean_CMEMS = Umean_CMEMS,
         Vmean_CMEMS = Vmean_CMEMS,
         Tmean_CMEMS = Tmean_CMEMS)